In [1]:
import tkinter
from tkinter import *
from tkinter import filedialog 
import pandas as pd
import numpy as np

import time

def edit_distance(s1, s2):
    m = len(s1)
    n = len(s2)

    # initialize the matrix
    d = [[0] * (n + 1) for i in range(m + 1)]

    # fill the first row and first column
    for i in range(1, m + 1):
        d[i][0] = i
    for j in range(1, n + 1):
        d[0][j] = j

    # fill the rest of the matrix
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s1[i - 1] == s2[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1

    # return the final value in the matrix
    return d[m][n]

 
def count_duplicates(records):
    count_dict = {}

    for row in records:
        record = tuple(row[1:]) # discard column 1

        if record in count_dict:
            count_dict[record] += 1
        else:
            count_dict[record] = 1

    duplicates = []
              
    for record, count in count_dict.items():
        if count > 1:
            duplicates.append(f"{record}: duplicate count: {count} ")
    return duplicates



In [4]:
def upload_file():
    file=filedialog.askopenfilename(title="Select file" , filetypes=[("All files", '*.*'),("CSV files", '.csv')])

    if(file):
        lbl_my_file=Label(window, text="File read!", font=("arial", 12, "bold") )
        lbl_my_file.pack()
        
        comparison=0
        threshold=10
        chunk_size=50
        
        canvas = None
        probable=[]
        matching_ids=[]
        
        
        df = pd.read_csv(file, infer_datetime_format=True, encoding='ISO-8859-1')
        
        file_name.set(file)
        df["name"] = df["name"].str.replace(" ", "")
        std = df[['id', 'name', 'age', 'age_unit', 'province_id', 'district_id', 'municipality_id', 'tole', 'ward', 'caste', 'sex']].values
#         print(std)
        df['std_att'] = df.apply(lambda row: str(row['name']) + str(row['age']) + str(row['age_unit']) + str(row['province_id']) + str(row['district_id']) + str(row['municipality_id']) + str(row['tole']) + str(row['ward']) + str(row['caste']) + str(row['sex']), axis=1)
#         print(df['std_att'])
        std_att = df['name'].astype(str) + df['age'].astype(str) + df['age_unit'].astype(str) + df['province_id'].astype(str) + df['district_id'].astype(str) + df['municipality_id'].astype(str) + df['tole'].astype(str) + df['ward'].astype(str) + df['caste'].astype(str) + df['sex'].astype(str)
       
        for index, row in df.iterrows():
            a=f"ID: {row['id']}, std_att: {row['std_att']}"
            print(a)
    
    
    
        mylist = Text(window)
        mylist.pack(side=LEFT, fill=BOTH, expand=True)

        # create a scrollbar and associate it with the listbox
        sb = Scrollbar(window)
        sb.pack(side=RIGHT, fill=Y)
        mylist.config(yscrollcommand=sb.set)
        sb.config(command=mylist.yview)
        
        unique_ids = [] # list to store unique IDs
        unique_std_att = [] # list to store corresponding unique std_att values

        for i in range(0, len(df), chunk_size):
            chunk = df.iloc[i:i+chunk_size]
            std_chunk=std_att.iloc[i:i+chunk_size]
            print(type(chunk))  # add this line to check the data type
            m = len(chunk)
            for j in range(m):
                for k in range(j + 1, m):
                    id_j = str(chunk.iloc[j]['id'])
                    id_k = str(chunk.iloc[k]['id'])
                    dist = edit_distance(std_chunk.iloc[j], std_chunk.iloc[k])
                    comparison += 1 # m * (m - 1) / 2
                    if dist <= threshold:
                        pair = (dist, f"Edit distance between ({id_j}){chunk.iloc[j]['std_att']} and ({id_k}){chunk.iloc[k]['std_att']} : {dist}")
                        probable.append(pair)    
                        
                        if id_j not in unique_ids:
                            unique_ids.append(id_j)
                            unique_std_att.append((chunk.iloc[j]['id'], chunk.iloc[j]['std_att']))
                        if id_k not in unique_ids:
                            unique_ids.append(id_k)
                            unique_std_att.append((chunk.iloc[k]['id'], chunk.iloc[k]['std_att']))
                            
                            
        probable = sorted(probable, key=lambda x: x[0]) # sort by distance
        for pair in probable:
            print(pair[1])
                
                
        print("attribute with unique id")
        print(unique_std_att)
        print("Count: ", len(unique_std_att))
        
        print("duplicates: ")
        duplicates = count_duplicates(unique_std_att)
        
#         for duplicate in duplicates:
#             print(f"{len(duplicate)} duplicates found ")
#             for std_att in unique_std_att:
#                 if std_att[1] == duplicate[1]:
#                     print(f"  ID: {std_att[0]}")
# 
        
        for i in range(len(duplicates)):
            
            print(f"{duplicates[i]}\n")
            mylist.insert(END,f"ID: {str(chunk.iloc[i]['id'])}: {duplicates[i]}")
        print("Duplicate count: ", len(duplicates))  
                    
        lbl_data=Label(window, text="Number of data in dataset:")
        lbl_data.pack()
        lbl_data_count=Label(window,text=len(std_att))
        lbl_data_count.pack()            
        lbl_comparison=Label(window, text="Number of Comparison")
        lbl_comparison.pack()            
        lbl=Label(window,text=comparison)
        lbl.pack()
        

        # create scrollbar
        sb = Scrollbar(window)
        sb.pack(side=BOTTOM, fill=BOTH)
        
        window.update_idletasks()
        canvas.config(scrollregion=canvas.bbox("all"))
#         end_time = time.time()
#         print(f"Total time taken: {end_time - start_time} seconds")




In [5]:
window=Tk()
window.geometry("1920x1080")
window.title("Data Redundancy Detection System")
title_lbl=Label(window, text="Upload .csv file", font=("arial", 30, "italic bold"),bd=7)
title_lbl.pack()
button=Button(window, text="Upload File", relief=RAISED,font=("arial", 15, "bold"), width=15, bg="light blue" ,command=upload_file)
button.pack()

file_name=StringVar()
file_data=StringVar()
my_file=StringVar()

lbl_name=Label(window, textvariable=file_name)
lbl_name.pack()


window.mainloop()


ID: 121, std_att: TekBahadurBista280769680Bajuda241
ID: 122, std_att: TekBahadurBista280769680Bajuda241
ID: 125, std_att: PratimaNeupane190329314Churkharka442
ID: 142, std_att: BalBd.Sarki330771705khalla401
ID: 143, std_att: DipakSinghSarki440771705khalla401
ID: 144, std_att: ShreeKoli320771705khalla401
ID: 146, std_att: GaurabBahadurBist240772717jhalari1041
ID: 147, std_att: KrishaSharki20771703jagatpur602
ID: 148, std_att: NarayanPrasadTimilsena330769680kuika541
ID: 151, std_att: sumanChapagai280321239manthali141
ID: 180, std_att: BishnuPrasadTimilsena330769680Kuika541
ID: 181, std_att: PuranKarki300769680Bayala341
ID: 182, std_att: ParashBista200769680Muli141
ID: 183, std_att: JivanRajUpadhyay350769679Ghodasain841
ID: 184, std_att: TapendraUpadhyay320769679Lekhgaun841
ID: 185, std_att: DhurbaBista180769680Muli141
ID: 186, std_att: ParashBista190769680Muli141
ID: 187, std_att: KhemBista190769680Muli141
ID: 188, std_att: LaxmanBista180769680Muli141
ID: 189, std_att: HikmatBista1707696

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\Dell\AppData\Local\Temp\ipykernel_10544\709757211.py", line 108, in upload_file
    canvas.config(scrollregion=canvas.bbox("all"))
AttributeError: 'NoneType' object has no attribute 'config'
